### 데이터분석 파일

##### feature 설명
- 'Bib':  단순 id / 동명이인 등을 구분하기 위함
- 'Age_group' → 19: 19세 이하 / 70: 70세 이상 / 20~69세의 경우 → 29: 25~29세
- ‘M/F’ → Man = 0 / Woman = 1 
- 'Country' = 국적
- '5p' → 출발로부터 5km까지 기준 pace[sec/km](sec)
- '15p' → 10km부터 15km까지 기준 pace[sec/km](sec)
- 'Final_Time’ → 풀코스(42.195km) 완주 기록(sec) 
- ‘Sub’ → sub-3,4,5,6,7을 숫자로만 표현 / 3 = 완주기록 3시간 미만(sec)
- '5k' → 출발로부터 5km되었을때의 시간 기록(sec)
- '15k' → 출발로부터 15km되었을때의 시간 기록(sec)
- ‘Dataset’ → 사용 데이터셋 구분 [B: 보스턴 마라톤 / C: 시카고 / M: 모스크바]
- ‘Year’ → 해당 데이터의 년도

In [4]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv('./data/combined_marathons_with_weather.csv')

#5개 랜덤 샘플
sample = df.sample(n=5, random_state=42)
print("=== 5개 랜덤 샘플 ===")
print(sample)


=== 5개 랜덤 샘플 ===
         Bib  Age_group  M/F Country   5p  10p  15p    20p  25p  30p  ...  \
69213  69214         70    0     USA  328  326  326  328.0  330  335  ...   
6925    6926         59    0     USA  268  269  271  273.0  275  277  ...   
67420  67421         39    1     USA  284  295  305  312.0  323  328  ...   
11872  11873         59    0     USA  287  283  283  286.0  292  300  ...   
34369  34370         44    0     USA  252  249  250  256.0  270  276  ...   

        15K     20K   25K    30K    35K    40K  Dataset  Year  \
69213  4903  6568.0  8258  10057  11882  13804        B  2017   
6925   4068  5472.0  6876   8319   9792  11293        B  2015   
67420  4578  6248.0  8075   9843  11681  13431        B  2017   
11872  4252  5732.0  7307   9028  10628  12222        B  2015   
34369  3757  5122.0  6750   8309  10123  11901        B  2016   

       temperature_race  humidity_race  
69213           21.0000         39.500  
6925             7.0875         87.750  
67420 

In [5]:
#기초 통계량 (count, mean, std, min, 25%, 50%, 75%, max)
print("\n=== 기초 통계량(summary) ===")
print(df.describe())


=== 기초 통계량(summary) ===
                Bib     Age_group           M/F            5p           10p  \
count  87668.000000  87668.000000  87668.000000  87668.000000  87668.000000   
mean   43834.500000     43.890975      0.426347    307.989335    308.020190   
std    25307.716037     11.386537      0.494548     48.638794     49.000235   
min        1.000000     19.000000      0.000000    176.000000      6.000000   
25%    21917.750000     34.000000      0.000000    273.000000    273.000000   
50%    43834.500000     44.000000      0.000000    302.000000    301.000000   
75%    65751.250000     54.000000      1.000000    336.000000    335.000000   
max    87668.000000     70.000000      1.000000    804.000000    643.000000   

                15p           20p           25p           30p           35p  \
count  87668.000000  79057.000000  87668.000000  87668.000000  87668.000000   
mean     309.905724    310.100004    319.226719    323.894192    329.855569   
std       50.530835     50

In [24]:
# 3) 전체 피처·타깃 결측 확인 (optional)
print("\n=== 전체 데이터셋 결측 개수 ===")
print(df.isna().sum())


=== 전체 데이터셋 결측 개수 ===
Bib                    0
Age_group              0
M/F                    0
Country                0
5p                     0
10p                    0
15p                    0
20p                 8611
25p                    0
30p                    0
35p                    0
40p                    0
Final_Time             0
Sub                    0
5K                     0
10K                    0
15K                    0
20K                 8611
25K                    0
30K                    0
35K                    0
40K                    0
Dataset                0
Year                   0
temperature_race       0
humidity_race          0
wt_5K                  0
wt_10K                 0
wt_15K                 0
wt_20K                 0
wt_25K                 0
wt_30K                 0
wt_35K                 0
wt_40K                 0
dtype: int64


In [20]:
import pandas as pd

# 1) 데이터 불러오기
df = pd.read_csv('./data/combined_marathons_with_weather.csv')

# 2) 보스턴 구간 경계점 고도 (m)
boundary_alt = {
    0.0: 149,
    5.0:  72,
    10.0: 55,
    15.0: 46,
    20.0: 46,
    25.0: 40,
    30.0: 18,
    35.0: 40,
    40.0: 18,
}

# 3) 5km 구간별 순고도 변화 계산
seg_ends = [5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0]
seg_weights = {
    end: boundary_alt[end] - boundary_alt[end - 5.0]
    for end in seg_ends
}

# 4) 컬럼 초기화 (0)
for end in seg_ends:
    df[f'wt_{int(end)}K'] = 0

# 5) 보스턴 행에만 가중치 채우기
mask = df['Dataset'] == 'B'
for end, w in seg_weights.items():
    df.loc[mask, f'wt_{int(end)}K'] = w

# 6) 결과 확인
print(df.loc[mask, [f'wt_{int(e)}K' for e in seg_ends]].head())

# 보스턴이 아닌(non-B) 행 중에서 5개 랜덤 샘플
non_b_sample = df[df['Dataset'] != 'B'].sample(n=5, random_state=42)

print("=== 5개 랜덤 샘플 ===")
print(non_b_sample)


   wt_5K  wt_10K  wt_15K  wt_20K  wt_25K  wt_30K  wt_35K  wt_40K
0    -77     -17      -9       0      -6     -22      22     -22
1    -77     -17      -9       0      -6     -22      22     -22
2    -77     -17      -9       0      -6     -22      22     -22
3    -77     -17      -9       0      -6     -22      22     -22
4    -77     -17      -9       0      -6     -22      22     -22
=== 5개 랜덤 샘플 ===
         Bib  Age_group  M/F Country   5p  10p  15p  20p  25p  30p  ...  \
80483  80484         29    0     RUS  361  368  387  NaN  468  482  ...   
80937  80938         34    0     RUS  318  307  298  NaN  320  302  ...   
87080  87081         44    0     JPN  409  391  394  NaN  448  457  ...   
81741  81742         59    0     RUS  412  419  427  NaN  559  555  ...   
85647  85648         44    0     RUS  383  397  397  NaN  416  439  ...   

       temperature_race  humidity_race  wt_5K  wt_10K  wt_15K  wt_20K  wt_25K  \
80483           11.0875         81.375      0       0       0

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score

# 2) Sub-3 검증 데이터 확보 (예: Sub == 3)
df_sub3 = df[df['Sub'] == 3].reset_index(drop=True)

# 3) 특성 및 타깃 정의
#   X: 신체정보, 목표그룹(Sub), 환경요인, 보스턴 고도 가중치(wt_*)
feature_cols = [
    'M/F',          # 성별 (0=남,1=여)
    'Age_group',    # 연령대
    'Sub',          # 목표 그룹
    'temperature_race',
    'humidity_race',
    # 보스턴만 값이 들어있는 wt_컬럼 (없으면 NaN→0 처리 필요)
    'wt_5K', 'wt_10K', 'wt_15K', 'wt_20K',
    'wt_25K', 'wt_30K', 'wt_35K', 'wt_40K'
]

#   Y: 5km 구간별 페이스(초/km)
target_cols = ['5p','10p','15p','20p','25p','30p','35p','40p']

#-------------------------------------------

# 1) 컬럼별 결측치 개수 확인
print("=== 타깃 컬럼별 NaN 개수 ===")
print(df_sub3[target_cols].isna().sum())

# 2) 적어도 하나의 타깃이 NaN인 행 보기
nan_rows = df_sub3[df_sub3[target_cols].isna().any(axis=1)]
print(f"\n=== NaN이 있는 행({len(nan_rows)}건) 샘플 ===")
print(nan_rows.head())

# 3) 전체 피처·타깃 결측 확인 (optional)
print("\n=== 전체 데이터셋 결측 개수 ===")
print(df_sub3.isna().sum())

#-------------------------------------------

X = df_sub3[feature_cols].fillna(0)   # NaN 이 있으면 0 으로 대체
Y = df_sub3[target_cols]

# 4) 학습/검증 분할
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

# 5) 파이프라인 구성 (스케일링 + 다중출력 회귀)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', MultiOutputRegressor(
        RandomForestRegressor(n_estimators=100, random_state=42)
    ))
])

# 6) 학습
pipeline.fit(X_train, Y_train)

# 7) 검증
Y_pred = pipeline.predict(X_val)

print("=== 구간별 MAE & R² ===")
for i, col in enumerate(target_cols):
    mae = mean_absolute_error(Y_val.iloc[:, i], Y_pred[:, i])
    r2  = r2_score(Y_val.iloc[:, i],    Y_pred[:, i])
    print(f"{col}: MAE={mae:.2f} sec/km,  R²={r2:.3f}")

# 8) 예측 예시: 30세 남성, Sub-3, 10℃·80% 습도, 보스턴 고도 가중치 적용
example = pd.DataFrame([{
    'M/F': 0,
    'Age_group': 30,
    'Sub': 3,
    'temperature_race': 10,
    'humidity_race': 80,
    'wt_5K':  -77,
    'wt_10K': -17,
    'wt_15K':  -9,
    'wt_20K':   0,
    'wt_25K': -6,
    'wt_30K': -22,
    'wt_35K': +22,
    'wt_40K': -22
}])
pred = pipeline.predict(example)
print("\n예측된 구간별 페이스(sec/km):")
print(dict(zip(target_cols, pred[0])))


=== 타깃 컬럼별 NaN 개수 ===
5p       0
10p      0
15p      0
20p    299
25p      0
30p      0
35p      0
40p      0
dtype: int64

=== NaN이 있는 행(299건) 샘플 ===
         Bib  Age_group  M/F Country   5p  10p  15p  20p  25p  30p  ...  \
79057  79058         34    1     RUS  211  211  207  NaN  221  212  ...   
79058  79059         39    1     RUS  211  211  214  NaN  233  224  ...   
79059  79060         34    0     RUS  189  190  193  NaN  202  198  ...   
79060  79061         39    0     RUS  189  186  187  NaN  207  205  ...   
79061  79062         44    0     RUS  195  196  195  NaN  214  213  ...   

       temperature_race  humidity_race  wt_5K  wt_10K  wt_15K  wt_20K  wt_25K  \
79057           11.0875         81.375      0       0       0       0       0   
79058           11.0875         81.375      0       0       0       0       0   
79059           11.0875         81.375      0       0       0       0       0   
79060           11.0875         81.375      0       0       0       0     

ValueError: Input y contains NaN.